In [1]:
''' 
https://www.dream11.com/games/point-system

ETL --> Extract Transform Load

OLTP (Database) --> Running database (Live database)
OLAP (Warehouse) --> Online Analytical Process (OLAP copy of OLTP so that people can with database)

4 steps
    - Extract data from AWS
    - Perform transformation
    - Make new database on AWS
    - Upload on AWS
'''

' \nhttps://www.dream11.com/games/point-system\n\nETL --> Extract Transform Load\n\nOLTP (Database) --> Running database (Live database)\nOLAP (Warehouse) --> Online Analytical Process (OLAP copy of OLTP so that people can with database)\n\n4 steps\n    - Extract data from AWS\n    - Perform transformation\n    - Make new database on AWS\n    - Upload on AWS\n'

In [2]:
import numpy as np
import pandas as pd
import requests
import mysql.connector

In [3]:
delivery = pd.read_csv("./deliveries.csv")
player = pd.read_csv("./Player.csv")
player_captain = pd.read_csv("./Player_Match.csv")

In [4]:
temp_df = player.merge(player_captain, on = 'Player_Id')[['Player_Name', 'Match_Id', 'Is_Captain']]

In [5]:
temp_df
delivery

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260916,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260917,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260918,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


In [6]:
delivery = delivery.merge(temp_df, left_on=['match_id', 'batter'], right_on=['Match_Id', 'Player_Name'], how='left').fillna(0)

In [7]:
runs = delivery.groupby(['match_id', 'batter'])['batsman_runs'].sum().reset_index()
balls = delivery.groupby(['match_id', 'batter'])['batsman_runs'].count().reset_index()

In [8]:
fours = delivery[delivery['batsman_runs'] == 4].groupby(['match_id', 'batter'])['batsman_runs'].count().reset_index()
sixes = delivery[delivery['batsman_runs'] == 6].groupby(['match_id', 'batter'])['batsman_runs'].count().reset_index()

In [9]:
runs.merge(balls, on = ['match_id', 'batter'])

# suffixes use for better col name after merge
final_df = runs.merge(balls, on = ['match_id', 'batter'], suffixes=('_runs', '_balls')).merge(fours, on = ['match_id', 'batter'], how = 'left').merge(sixes, on = ['match_id', 'batter'], how = 'left')

In [10]:
final_df = final_df.fillna(0)

In [11]:
final_df = final_df.rename(columns={
    'batsman_runs_runs': 'runs',
    'batsman_runs_balls': 'balls',
    'batsman_runs_x': 'fours',
    'batsman_runs_y': 'sixes',
})

In [12]:
final_df['sr'] = round((final_df['runs'] / final_df['balls']) * 100)
final_df

,match_id,batter,runs,balls,fours,sixes,sr
0,335982,AA Noffke,9,12,1.0,0.0,75.0
1,335982,B Akhil,0,2,0.0,0.0,0.0
2,335982,BB McCullum,158,77,10.0,13.0,205.0
3,335982,CL White,6,10,0.0,0.0,60.0
4,335982,DJ Hussey,12,12,1.0,0.0,100.0
...,...,...,...,...,...,...,...
16510,1426312,SP Narine,6,2,0.0,1.0,300.0
16511,1426312,SS Iyer,6,3,1.0,0.0,200.0
16512,1426312,Shahbaz Ahmed,8,7,0.0,1.0,114.0
16513,1426312,TM Head,0,1,0.0,0.0,0.0


In [13]:
final_df = final_df.merge(temp_df, left_on=['match_id', 'batter'], right_on=['Match_Id', 'Player_Name'], how = 'left').drop(columns=['Player_Name', 'Match_Id']).fillna(0)

In [14]:
final_df

,match_id,batter,runs,balls,fours,sixes,sr,Is_Captain
0,335982,AA Noffke,9,12,1.0,0.0,75.0,0.0
1,335982,B Akhil,0,2,0.0,0.0,0.0,0.0
2,335982,BB McCullum,158,77,10.0,13.0,205.0,0.0
3,335982,CL White,6,10,0.0,0.0,60.0,0.0
4,335982,DJ Hussey,12,12,1.0,0.0,100.0,0.0
...,...,...,...,...,...,...,...,...
16510,1426312,SP Narine,6,2,0.0,1.0,300.0,0.0
16511,1426312,SS Iyer,6,3,1.0,0.0,200.0,0.0
16512,1426312,Shahbaz Ahmed,8,7,0.0,1.0,114.0,0.0
16513,1426312,TM Head,0,1,0.0,0.0,0.0,0.0


In [15]:
def make_dream11(row):
    score = 0
    score += row['runs'] + row['fours'] + row['sixes'] * 2

    if row['runs'] >= 30:
        score += 4
    if row['runs'] >= 50:
        score += 8 
    if row['runs'] >= 100:
        score += 16
    if row['runs'] == 0:
        score -= 2

    if row['balls'] >= 10:
        if row['sr'] > 170:
            score += 6
        elif 150.01 <= row['sr'] <= 170:
            score += 4
        elif 130 <= row['sr'] <= 150:
            score += 2
        elif 60 <= row['sr'] <= 70:
            score -= 2
        elif 50 <= row['sr'] < 60:
            score -= 4
        elif row['sr'] < 50:
            score -= 6
    
    if row['Is_Captain'] == 1:
        score *= 2
    
    return score

In [16]:
final_df['dream11_point'] = final_df.apply(make_dream11, axis=1)

In [17]:
final_df

,match_id,batter,runs,balls,fours,sixes,sr,Is_Captain,dream11_point
0,335982,AA Noffke,9,12,1.0,0.0,75.0,0.0,10.0
1,335982,B Akhil,0,2,0.0,0.0,0.0,0.0,-2.0
2,335982,BB McCullum,158,77,10.0,13.0,205.0,0.0,228.0
3,335982,CL White,6,10,0.0,0.0,60.0,0.0,4.0
4,335982,DJ Hussey,12,12,1.0,0.0,100.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...
16510,1426312,SP Narine,6,2,0.0,1.0,300.0,0.0,8.0
16511,1426312,SS Iyer,6,3,1.0,0.0,200.0,0.0,7.0
16512,1426312,Shahbaz Ahmed,8,7,0.0,1.0,114.0,0.0,10.0
16513,1426312,TM Head,0,1,0.0,0.0,0.0,0.0,-2.0


In [18]:

export_df = final_df.sort_values(['dream11_point'], ascending=False)[['match_id', 'batter', 'dream11_point']]

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()
host = os.getenv('user_host')
username = os.getenv('user_name')
password = os.getenv('user_password')

conn = mysql.connector.connect(host=host, user=username, password=password)

In [20]:
import pymysql
from sqlalchemy import create_engine

In [21]:
myCursor = conn.cursor()
# myCursor.execute('CREATE DATABASE dream11')
myCursor.execute('DROP DATABASE IF EXISTS dream11')
myCursor.execute('CREATE DATABASE dream11')
conn.commit()

In [22]:
# {root}:{password}@{url}/{database}
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}/dream11"
)

export_df.to_sql('batter_points', con = engine, if_exists='append')

16515

In [23]:
# -------------------------- This work done by Data Scientist

conn = mysql.connector.connect(host=host, user=username, password=password, database = 'dream11')
myCursor = conn.cursor()

In [24]:
pd.read_sql("SELECT * FROM batter_points", conn, index_col='index')

C:\Users\Asus\AppData\Local\Temp\ipykernel_11288\854628970.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM batter_points", conn, index_col='index')


,match_id,batter,dream11_point
index,,,
3325,501243,V Sehwag,356.0
2844,501210,SR Tendulkar,296.0
5302,598027,CH Gayle,256.0
2,335982,BB McCullum,228.0
4254,548342,V Sehwag,210.0
...,...,...,...
3546,501258,ND Doshi,-8.0
10432,1175356,SR Watson,-8.0
3648,501266,S Badrinath,-8.0
